# 한국어의 특징

## 한국어는 문법상 동사의 활용이 난해하고 조사의 존재 때문에 고립어인 영어보다 형태소 단위로 AI에게 학습시키기 난해하다.
## 하지만 영어보다는 문자를 발음할 때의 규칙성이 더 높기 때문에 STT/TTS를 제작할 때는 음절 단위로 학습시키는게 더 유리하다.


# 이를 위한 함수 chr(), ord()
## chr() : 문자 하나를 유니코드 숫자 값으로 변환시키는 함수
#### 예1) chr('A') == 65
#### 예2) chr('가') == 0xAC00
## ord() : 숫자를 해당 유니코드 숫자 값을 가진 문자 하나로 변환시키는 함수
#### 예1) ord(97) == 'a'


# 한국어 유니코드 완성형의 특징
## 한국어 유니코드는 완성형으로 숫자 값이 배정되어 있으나 숫자값 배정에 규칙이 존재해서 초성, 중성, 종성 분리가 쉽다
## 공식 : (초성 * 21 + 중성) * 28 + 종성 + 0xAC00
#### 초성 :
['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (19개)
#### 중성 :
['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ'] (21개)
#### 종성 : 
['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ'] (28개)
#### 기타 특문을 위한 라벨링 :
['unk',' ','.','!','?'] (5개, 가변적)


## 이를 위한 Multi-hot-encoding
### 기존 One-hot-encoding과 달리 1의 값이 여러개인 인코딩 방식
#### 예시로 '건'이라는 글자는
#### 앞의 0~18번째 벡터 중 0번째가 1, 나머지가 0
#### 중간의 19~39번째 백터 중 23번째가 1, 나머지가 0
#### 뒤의 40~67번째 벡터 중 44번째가 1, 나머지가 0
#### 위와 같이 표현이 가능하다






참고 문헌 : https://koreascience.or.kr/article/CFKO201832073079068.pdf

In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(""), ".."))

import custom
import numpy
import pickle

In [2]:


cho_list = ['ㄱ','ㄲ','ㄴ','ㄷ','ㄸ','ㄹ','ㅁ','ㅂ','ㅃ','ㅅ','ㅆ','ㅇ','ㅈ','ㅉ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
jung_list = ['ㅏ','ㅐ','ㅑ','ㅒ','ㅓ','ㅔ','ㅕ','ㅖ','ㅗ','ㅘ','ㅙ','ㅚ','ㅛ','ㅜ','ㅝ','ㅞ','ㅟ','ㅠ','ㅡ','ㅢ','ㅣ']
jong_list = ['','ㄱ','ㄲ','ㄳ','ㄴ','ㄵ','ㄶ','ㄷ','ㄹ','ㄺ','ㄻ','ㄼ','ㄽ','ㄾ','ㄿ','ㅀ','ㅁ','ㅂ','ㅄ','ㅅ','ㅆ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ']
spec_list = ['<unk>',' ','.','!','?']

print(len(cho_list))
print(len(jung_list))
print(len(jong_list))
print(len(spec_list))

total_len = len(cho_list)+len(jung_list)+len(jong_list)+len(spec_list)
print(total_len)

19
21
28
5
73


In [3]:
# 음절-벡터 사전 만들기

vec_dict = {}
vec_dict["<pad>"] = numpy.zeros(total_len) #pad는 영행렬 고정

for i in range(len(spec_list)) :
    vec = numpy.zeros(total_len)
    vec[68 + i] += 1

    vec_dict[spec_list[i]] = vec

for i in range(ord('가'), ord('힣') + 1) :
    vec = numpy.zeros(total_len)
    val = i - 0xAC00
    
    chosung  = (val // 28) // 21
    jungsung = (val // 28) % 21
    jongsung = val % 28
    
    vec[chosung] += 1
    vec[jungsung] += 1
    vec[jongsung] += 1

    vec_dict[chr(i)] = vec

print(len(vec_dict))

11178


In [4]:
# 음절-벡터 사전 저장

with open("k_sound_vec.pkl", mode = "wb") as f:
    pickle.dump(vec_dict, f)

In [6]:
# 음절과 벡터 분리해서 저장
sound_dict = {}
for i in range(len(vec_dict)) :
    sound_dict[list(vec_dict.keys())[i]] = i

vec_list = numpy.array(list(vec_dict.values()))

print(sound_dict)
print(vec_list)

with open("k_sound_dict.pkl", mode = "wb") as f:
    pickle.dump(sound_dict, f)

with open("k_vec_list.pkl", mode = "wb") as f:
    pickle.dump(vec_list, f)
    

{'<pad>': 0, '<unk>': 1, ' ': 2, '.': 3, '!': 4, '?': 5, '가': 6, '각': 7, '갂': 8, '갃': 9, '간': 10, '갅': 11, '갆': 12, '갇': 13, '갈': 14, '갉': 15, '갊': 16, '갋': 17, '갌': 18, '갍': 19, '갎': 20, '갏': 21, '감': 22, '갑': 23, '값': 24, '갓': 25, '갔': 26, '강': 27, '갖': 28, '갗': 29, '갘': 30, '같': 31, '갚': 32, '갛': 33, '개': 34, '객': 35, '갞': 36, '갟': 37, '갠': 38, '갡': 39, '갢': 40, '갣': 41, '갤': 42, '갥': 43, '갦': 44, '갧': 45, '갨': 46, '갩': 47, '갪': 48, '갫': 49, '갬': 50, '갭': 51, '갮': 52, '갯': 53, '갰': 54, '갱': 55, '갲': 56, '갳': 57, '갴': 58, '갵': 59, '갶': 60, '갷': 61, '갸': 62, '갹': 63, '갺': 64, '갻': 65, '갼': 66, '갽': 67, '갾': 68, '갿': 69, '걀': 70, '걁': 71, '걂': 72, '걃': 73, '걄': 74, '걅': 75, '걆': 76, '걇': 77, '걈': 78, '걉': 79, '걊': 80, '걋': 81, '걌': 82, '걍': 83, '걎': 84, '걏': 85, '걐': 86, '걑': 87, '걒': 88, '걓': 89, '걔': 90, '걕': 91, '걖': 92, '걗': 93, '걘': 94, '걙': 95, '걚': 96, '걛': 97, '걜': 98, '걝': 99, '걞': 100, '걟': 101, '걠': 102, '걡': 103, '걢': 104, '걣': 105, '걤': 106, '걥': 107, '걦': 108, '걧': 109, '

In [9]:
# 문장을 음절 라벨, 벡터로 테스트

text = "안녕하세요 반갑습니다."
sound = list(text)

sound_label = custom.word_vectorize(sound, sound_dict)
sound_vector = numpy.array(custom.word_vectorize(sound, vec_dict))
print(sound_label)
print(sound_vector)

[6478, 1371, 10590, 5438, 6810, 2, 4126, 23, 5819, 1742, 1770, 3]
[[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [1. 0. 0. 0. 0. 0. 0.

In [12]:
# 음절 라벨을 문장으로 테스트

num_dict = {v : k for k, v in sound_dict.items()}

re_text = custom.word_vectorize(sound_label, num_dict)
print(re_text)
print("".join(re_text))

['안', '녕', '하', '세', '요', ' ', '반', '갑', '습', '니', '다', '.']
안녕하세요 반갑습니다.
